In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from FAA import Ohare
from FAA import Hanger
import pandas as pd

faa = Ohare.Ohare(table_name='ord_w_wthr', sql=True\
                  , sql_user='ubuntu', host_name='18.188.180.59')
db = faa.get_modeling_df()
db.head()

,Quarter,Month,DayOfWeek,Distance,awnd,prcp,snow,snwd,tavg,9E,...,OH,OO,UA,VX,YX,DepTime_early_am,DepTime_early_pm,DepTime_late_am,DepTime_late_pm,Was_Delayed
0,1,2,2,733.0,13.87,0.0,0.0,0.0,48,0,...,0,1,0,0,0,0,1,0,0,False
3,1,2,2,474.0,13.87,0.0,0.0,0.0,48,0,...,0,1,0,0,0,0,1,0,0,False
5,1,2,2,733.0,13.87,0.0,0.0,0.0,48,0,...,0,1,0,0,0,0,0,1,0,False
8,1,2,2,84.0,13.87,0.0,0.0,0.0,48,0,...,0,1,0,0,0,0,0,0,1,False
10,1,2,2,911.0,13.87,0.0,0.0,0.0,48,0,...,0,1,0,0,0,0,1,0,0,True


In [2]:
db.groupby(db.Was_Delayed).count()

,Quarter,Month,DayOfWeek,Distance,awnd,prcp,snow,snwd,tavg,9E,...,NK,OH,OO,UA,VX,YX,DepTime_early_am,DepTime_early_pm,DepTime_late_am,DepTime_late_pm
Was_Delayed,,,,,,,,,,,,,,,,,,,,,
False,361852,361852,361852,361852,361852,361852,361852,361852,361852,361852,...,361852,361852,361852,361852,361852,361852,361852,361852,361852,361852
True,182560,182560,182560,182560,182560,182560,182560,182560,182560,182560,...,182560,182560,182560,182560,182560,182560,182560,182560,182560,182560


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import *

In [6]:
y = db.Was_Delayed
X = db.drop(columns=['Was_Delayed'])

In [7]:
X_traintest, X_holdout, y_traintest, y_holdout = train_test_split(X, y, random_state=51, test_size=.2)

## Random Forest

In [48]:
frst = RandomForestClassifier()

In [49]:
frst.fit(X_tr,y_tr)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
frst.score(X_val,y_val)

0.6542710983349099

In [51]:
frst.score(X, y)

0.8041409814625687

In [47]:
f1_score(y_val, frst.predict(X_val)) -> With some custom features selected that had the roc curve go up to .66
frst = RandomForestClassifier(#max_depth=np.log2(len(db.columns.tolist()))
                                                n_jobs=-1
                                                ,random_state=51
                                                ,min_samples_leaf=50
                                                ,max_depth=4  
                                            )

0.020658036677454156

In [52]:
f1_score(y_val, frst.predict(X_val))

0.44699729697966856

In [53]:
precision_score(y_val, frst.predict(X_val))

0.4833063312049303

In [55]:
for i in zip(X.columns, frst.feature_importances_):
    print (i)

('Quarter', 0.00916522339347238)
('Month', 0.03235523146713787)
('DayOfWeek', 0.04754666978454768)
('Distance', 0.568234609866671)
('awnd', 0.09173875024550922)
('prcp', 0.04544290567520066)
('snow', 0.006812201836613381)
('snwd', 0.006134099938468461)
('tavg', 0.09118199640351096)
('9E', 0.0012306239833580577)
('AA', 0.011198110584308116)
('AS', 0.002372980921996305)
('B6', 0.0031511378973689006)
('DL', 0.005119487307991593)
('EV', 0.005318785893093139)
('F9', 0.0031462972477306764)
('MQ', 0.005247243038418695)
('NK', 0.005719783313258358)
('OH', 0.0016026999819824315)
('OO', 0.008902223974525707)
('UA', 0.01100259938932166)
('VX', 0.0010469026322336302)
('YX', 0.003878126905132784)
('DepTime_early_am', 0.004415957792078188)
('DepTime_early_pm', 0.005298696511552472)
('DepTime_late_am', 0.013991499474212132)
('DepTime_late_pm', 0.008745154540305632)


#### Metrics

In [84]:
#from sklearn.metrics import *
def print_metrics(y_true, y_prediction):
    f1 = f1_score(y_true,y_prediction)
    presc = precision_score(y_true, y_prediction)
    auc = roc_auc_score(y_true, y_prediction)
    recl = recall_score(y_true, y_prediction)
    
    print(f'The F1 Score is: {f1}\n The Precision score is: {presc}\n The Recall Score is: {recl} \n The AUC score is {auc}'
         )
    print (confusion_matrix(y_true, y_prediction, labels=[True, False]))

In [85]:
print_metrics(y, frst.predict(X))

The F1 Score is: 0.6925339392611218
 The Precision score is: 0.7311673445529604
 The Recall Score is: 0.6577782646801051 
 The AUC score is 0.7678807642779719
[[120084  62476]
 [ 44152 317700]]


## Logistic Regression

In [67]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=2**-5)
lr.fit(X_tr,y_tr)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.03125, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [86]:
print_metrics(y_val, lr.predict(X_val))

The F1 Score is: 0.18791052317302834
 The Precision score is: 0.5887178011224636
 The Recall Score is: 0.11179733828874375 
 The AUC score is 0.5361310664330702
[[ 4091 32502]
 [ 2858 69432]]


# Naive Bayes

In [2]:
from sklearn.naive_bayes import GaussianNB



In [8]:
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score

In [15]:
metrics = ['roc_auc','f1', 'accuracy', 'precision', 'recall']

In [16]:
nb = GaussianNB()
cv = cross_validate(nb, X_traintest, y_traintest, n_jobs=-1, cv=10, scoring=metrics, return_train_score=True)

In [18]:
cv

{'fit_time': array([0.66254497, 0.7719481 , 0.80609202, 0.85577607, 0.91293573,
        0.94252014, 0.92884398, 1.0211668 , 1.02004409, 1.04460502]),
 'score_time': array([0.47212911, 0.44362116, 0.43498802, 0.4296639 , 0.42460513,
        0.42839789, 0.44037008, 0.49765921, 0.51600885, 0.47039485]),
 'test_roc_auc': array([0.62102303, 0.62098255, 0.62119262, 0.61954046, 0.62260339,
        0.6287913 , 0.62401051, 0.61938593, 0.62515529, 0.62317177]),
 'train_roc_auc': array([0.62336341, 0.62264737, 0.62278672, 0.62308935, 0.62254373,
        0.62218771, 0.62225037, 0.62360387, 0.62285373, 0.62330921]),
 'test_f1': array([0.35661463, 0.35532117, 0.3510879 , 0.34832145, 0.35653044,
        0.35372306, 0.36305842, 0.34704517, 0.35091853, 0.33769121]),
 'train_f1': array([0.35816332, 0.35548763, 0.35102274, 0.35192109, 0.35302761,
        0.34619014, 0.36005727, 0.35310281, 0.34920094, 0.34536862]),
 'test_accuracy': array([0.65842403, 0.65757451, 0.65761256, 0.65591348, 0.66052855,
     

In [28]:
corrs = faa.get_modeling_df().corr()

In [29]:
corrs.applymap(lambda x: c(x)).max()

Quarter             0.964617
Month               0.964617
DayOfWeek           0.019612
Distance            0.271923
awnd                0.060954
prcp                0.116624
snow                0.497701
snwd                0.497701
tavg                0.402120
9E                  0.015461
AA                  0.262063
AS                  0.197688
B6                  0.034730
DL                  0.020718
EV                  0.023203
F9                  0.066702
MQ                  0.039968
NK                  0.108150
OH                  0.007723
OO                  0.030639
UA                  0.271923
VX                  0.110850
YX                  0.021259
DepTime_early_am    0.093423
DepTime_early_pm    0.034311
DepTime_late_am     0.083618
DepTime_late_pm     0.109300
Was_Delayed         0.116624
dtype: float64

In [27]:
corrs

,Quarter,Month,DayOfWeek,Distance,awnd,prcp,snow,snwd,tavg,9E,...,OH,OO,UA,VX,YX,DepTime_early_am,DepTime_early_pm,DepTime_late_am,DepTime_late_pm,Was_Delayed
Quarter,1.000000,0.964617,0.005611,0.014551,-0.189347,0.015103,-0.153762,-0.196492,0.378871,-2.130745e-02,...,-0.008010,0.017878,0.036742,0.004809,-0.040644,0.006110,0.005493,0.004367,-0.012522,-1.661824e-02
Month,0.964617,1.000000,0.013176,0.016344,-0.192480,0.005524,-0.145452,-0.195372,0.402120,-2.278343e-02,...,-0.008912,0.019797,0.038235,0.006246,-0.042606,0.006643,0.003810,0.004663,-0.011227,-1.786447e-02
DayOfWeek,0.005611,0.013176,1.000000,0.019612,-0.020741,-0.059294,-0.027707,-0.035027,-0.027267,-3.780015e-03,...,-0.001603,0.000965,-0.006434,-0.000547,-0.002038,-0.008620,0.004213,-0.000739,-0.000754,-1.155986e-02
Distance,0.014551,0.016344,0.019612,1.000000,-0.003372,-0.001635,-0.005190,-0.008590,-0.000326,-2.404658e-02,...,-0.038818,-0.363888,0.271923,0.110850,-0.073773,0.041525,-0.035674,0.083618,-0.064321,6.713125e-02
awnd,-0.189347,-0.192480,-0.020741,-0.003372,1.000000,0.060954,0.022912,-0.006101,-0.215562,1.083969e-04,...,-0.004279,0.007647,-0.005255,0.005706,-0.009340,0.003484,0.000967,-0.003113,0.001060,9.957333e-03
prcp,0.015103,0.005524,-0.059294,-0.001635,0.060954,1.000000,0.044219,-0.014140,0.111609,1.615007e-03,...,0.002160,-0.001155,-0.001164,-0.003091,0.006080,0.007523,-0.019977,-0.011906,0.031191,1.166240e-01
snow,-0.153762,-0.145452,-0.027707,-0.005190,0.022912,0.044219,1.000000,0.497701,-0.252506,5.861961e-03,...,0.004467,-0.002247,-0.011029,-0.000143,0.011988,-0.007045,-0.019295,-0.019420,0.043355,5.864629e-02
snwd,-0.196492,-0.195372,-0.035027,-0.008590,-0.006101,-0.014140,0.497701,1.000000,-0.321580,7.801839e-03,...,0.007723,-0.005569,-0.016578,-0.000211,0.021259,-0.006016,-0.007965,-0.011624,0.022756,3.181367e-02
tavg,0.378871,0.402120,-0.027267,-0.000326,-0.215562,0.111609,-0.252506,-0.321580,1.000000,-7.750026e-03,...,0.001136,0.007265,0.020217,-0.009662,-0.003327,0.005002,-0.010686,0.006681,0.002611,7.422755e-03
9E,-0.021307,-0.022783,-0.003780,-0.024047,0.000108,0.001615,0.005862,0.007802,-0.007750,1.000000e+00,...,-0.003266,-0.032646,-0.032301,-0.003110,-0.009382,0.015461,-0.004486,0.009741,-0.010772,-5.789683e-07


In [33]:
faa.get_modeling_df().sum()

Quarter             1.304905e+06
Month               3.343820e+06
DayOfWeek           2.136864e+06
Distance            4.165643e+08
awnd                5.263623e+06
prcp                7.167263e+04
snow                3.513470e+04
snwd                1.079057e+05
tavg                2.992576e+07
9E                  1.670000e+03
AA                  1.171700e+05
AS                  4.700000e+03
B6                  4.575000e+03
DL                  1.556700e+04
EV                  3.122000e+04
F9                  5.392000e+03
MQ                  4.949700e+04
NK                  1.798500e+04
OH                  1.881000e+03
OO                  1.400530e+05
UA                  1.378550e+05
VX                  1.706000e+03
YX                  1.514100e+04
DepTime_early_am    1.318500e+04
DepTime_early_pm    1.925210e+05
DepTime_late_am     1.851840e+05
DepTime_late_pm     1.535220e+05
Was_Delayed         1.825600e+05
dtype: float64